In [1]:
# obtener_historico_ofertas_stock.py
# Script para replicar datos de ventas de STOCK y OFERTAS desde SQL Server a PostgreSQL usando psycopg2 y Prefect.

import os
import sys
import pandas as pd
import psycopg2 as pg2
from psycopg2.extras import execute_values
import logging
from prefect import flow, task, get_run_logger
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Configurar logging
logger = logging.getLogger("replicacion_logger")
logger.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
os.makedirs("logs", exist_ok=True)
file_handler = logging.FileHandler("logs/replicacion_psycopg2.log", encoding="utf-8")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# Cargar variables de entorno
load_dotenv()
SQL_SERVER = os.getenv("SQL_SERVER")
SQL_USER = os.getenv("SQL_USER")
SQL_PASSWORD = os.getenv("SQL_PASSWORD")
SQL_DATABASE = os.getenv("SQL_DATABASE")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DB = os.getenv("PG_DB")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")

# Crear engine SQL Server
def open_sql_conn():
    print(f"Conectando a SQL Server: {SQL_SERVER}")
    print(f"Conectando a SQL Server: {SQL_DATABASE}") 
    return create_engine(f"mssql+pyodbc://{SQL_USER}:{SQL_PASSWORD}@{SQL_SERVER}/{SQL_DATABASE}?driver=ODBC+Driver+17+for+SQL+Server")

def open_pg_conn():
    return pg2.connect(dbname=PG_DB, user=PG_USER, password=PG_PASSWORD, host=PG_HOST, port=PG_PORT)

def infer_postgres_types(df):
    type_map = {
        "int64": "BIGINT",
        "int32": "INTEGER",
        "float64": "DOUBLE PRECISION",
        "bool": "BOOLEAN",
        "datetime64[ns]": "TIMESTAMP",
        "object": "TEXT"
    }
    col_defs = [f"{col} {type_map.get(str(df[col].dtype), 'TEXT')}" for col in df.columns]
    return ", ".join(col_defs)


In [2]:
ids = 20
print(f"-> Generando datos de STOCK para ID: {ids}")

# Cargar STOCK por Proveedor
"""Consulta el stock y devuelve un dict {fecha: cantidad}, limitado a fechas válidas hasta ayer."""
conn = open_sql_conn()
query_stock = f"""
    SELECT  
        S.[C_ANIO],S.[C_MES],S.[C_SUCU_EMPR],S.[C_ARTICULO], A.[C_PROVEEDOR_PRIMARIO]
        ,S.[Q_DIA1],S.[Q_DIA2],S.[Q_DIA3],S.[Q_DIA4],S.[Q_DIA5],S.[Q_DIA6],S.[Q_DIA7],S.[Q_DIA8],S.[Q_DIA9],S.[Q_DIA10]
        ,S.[Q_DIA11],S.[Q_DIA12],S.[Q_DIA13],S.[Q_DIA14],S.[Q_DIA15],S.[Q_DIA16],S.[Q_DIA17],S.[Q_DIA18],S.[Q_DIA19],S.[Q_DIA20]
        ,S.[Q_DIA21],S.[Q_DIA22],S.[Q_DIA23],S.[Q_DIA24],S.[Q_DIA25],S.[Q_DIA26],S.[Q_DIA27],S.[Q_DIA28],S.[Q_DIA29],S.[Q_DIA30],S.[Q_DIA31]  
    FROM [repl].[T710_ESTADIS_STOCK] S
    LEFT JOIN [repl].[T050_ARTICULOS] A  
        ON S.C_ARTICULO = A.C_ARTICULO  
    WHERE C_ANIO = 2025
    AND A.C_PROVEEDOR_PRIMARIO IN ({ids});
"""
df_stock = pd.read_sql(query_stock, conn) # type: ignore
conn.dispose()  # Cerrar conexión SQL Server

if df_stock.empty:
    print(f"⚠️ No se encontraron datos de stock para el proveedor {id_proveedor} en el mes actual.") # type: ignore
    # return {}
df_stock['C_ANIO'] = df_stock['C_ANIO'].astype(int)
df_stock['C_MES'] = df_stock['C_MES'].astype(int)
df_stock['C_ARTICULO'] = df_stock['C_ARTICULO'].astype(int)
df_stock['C_SUCU_EMPR'] = df_stock['C_SUCU_EMPR'].astype(int)
df_stock['C_PROVEEDOR_PRIMARIO'] = df_stock['C_PROVEEDOR_PRIMARIO'].astype(int)

-> Generando datos de STOCK para ID: 20
Conectando a SQL Server: 10.54.200.92
Conectando a SQL Server: data-sync


In [3]:
import pandas as pd
from datetime import date, datetime, timedelta

# Función para verificar si la fecha es válida
def es_fecha_valida(anio, mes, dia):
    try:
        return date(anio, mes, dia)
    except ValueError:
        return None

# Suponiendo que df_oferta es tu DataFrame original
# Creamos un DataFrame vacío para ir apilando las filas transformadas
df_trf_stock = pd.DataFrame()

# Iteramos sobre las columnas de ofertas
for col in df_stock.columns:
    if col.startswith("Q_DIA"):
        # Extraemos el número del día de la columna (por ejemplo, 'M_OFERTA_DIA1' -> 1)
        dia = int(col.replace("Q_DIA", ""))
        
        # Copiamos los datos necesarios para este día
        df_tmp = df_stock[['C_ANIO', 'C_MES', 'C_SUCU_EMPR', 'C_PROVEEDOR_PRIMARIO', 'C_ARTICULO', col]].copy()
        df_tmp['DIA'] = dia
        
        # Creamos la fecha a partir del año, mes y día
        df_tmp['FECHA'] = df_tmp.apply(lambda row: es_fecha_valida(int(row['C_ANIO']), int(row['C_MES']), int(row['DIA'])), axis=1)
        # El valor float es el de la columna de stock diario, por ejemplo: df_tmp[col]
        # En este contexto, df_tmp[col] contiene valores float64 (por ejemplo, 0.0, 464.0, etc.)
        # Si quieres crear una columna con el valor de stock, puedes hacer:
        df_tmp['STOCK'] = df_tmp[col].astype(float)   
        
        # Filtramos fechas inválidas
        df_tmp = df_tmp.dropna(subset=['FECHA'])
        
        # Filtramos fechas futuras (solo fechas hasta el día anterior)
        df_tmp = df_tmp[df_tmp['FECHA'] <= (datetime.now().date() - timedelta(days=1))]

        # Creamos la columna de 'OFERTA' (1 si tiene oferta, 0 si no)
        df_tmp['OFERTA'] = df_tmp[col].apply(lambda x: 1 if x == 'S' else 0)
        
        # Ahora solo conservamos las columnas relevantes
        df_tmp = df_tmp[['C_PROVEEDOR_PRIMARIO','C_ARTICULO', 'C_SUCU_EMPR', 'FECHA', 'STOCK']]
        
        # Agregamos al DataFrame final
        df_trf_stock = pd.concat([df_trf_stock, df_tmp], ignore_index=True)

# Mostrar el resultado final
print(df_trf_stock.head())


   C_PROVEEDOR_PRIMARIO  C_ARTICULO  C_SUCU_EMPR       FECHA  STOCK
0                    20         166            1  2025-01-01    0.0
1                    20        3796            1  2025-01-01    0.0
2                    20        3797            1  2025-01-01    0.0
3                    20        4770            1  2025-01-01    0.0
4                    20        4771            1  2025-01-01    0.0


In [4]:
print(df_tmp.head())

   C_PROVEEDOR_PRIMARIO  C_ARTICULO  C_SUCU_EMPR       FECHA  STOCK
0                    20         166            1  2025-01-31    0.0
1                    20        3796            1  2025-01-31  722.0
2                    20        3797            1  2025-01-31  453.0
3                    20        4770            1  2025-01-31   43.0
4                    20        4771            1  2025-01-31    0.0


In [6]:
# Cargar OFERTAS por Proveedor
"""Consulta el OFERTAS y devuelve un dict {fecha: flag}, limitado a fechas válidas hasta ayer."""
conn = open_sql_conn()
query_oferta = f"""
    SELECT  
        O.C_ANIO, O.C_MES, O.C_SUCU_EMPR, O.C_ARTICULO, A.C_PROVEEDOR_PRIMARIO,
        O.M_OFERTA_DIA1, O.M_OFERTA_DIA2, O.M_OFERTA_DIA3, O.M_OFERTA_DIA4, O.M_OFERTA_DIA5, 
        O.M_OFERTA_DIA6, O.M_OFERTA_DIA7, O.M_OFERTA_DIA8, O.M_OFERTA_DIA9, O.M_OFERTA_DIA10,
        O.M_OFERTA_DIA11, O.M_OFERTA_DIA12, O.M_OFERTA_DIA13, O.M_OFERTA_DIA14, O.M_OFERTA_DIA15, 
        O.M_OFERTA_DIA16, O.M_OFERTA_DIA17, O.M_OFERTA_DIA18, O.M_OFERTA_DIA19, O.M_OFERTA_DIA20,
        O.M_OFERTA_DIA21, O.M_OFERTA_DIA22, O.M_OFERTA_DIA23, O.M_OFERTA_DIA24, O.M_OFERTA_DIA25, 
        O.M_OFERTA_DIA26, O.M_OFERTA_DIA27, O.M_OFERTA_DIA28, O.M_OFERTA_DIA29, O.M_OFERTA_DIA30,
        O.M_OFERTA_DIA31
    FROM [repl].[T710_ESTADIS_OFERTA_FOLDER] O
    LEFT JOIN [repl].[T050_ARTICULOS] A ON O.C_ARTICULO = A.C_ARTICULO
    WHERE C_ANIO = 2025
    AND A.C_PROVEEDOR_PRIMARIO IN ({ids});
"""
df_oferta = pd.read_sql(query_oferta, conn) # type: ignore
conn.dispose()  # Cerrar conexión SQL Server
if df_stock.empty:
    print(f"⚠️ No se encontraron datos de stock para el proveedor {id_proveedor} en el mes actual.") # type: ignore
    # return {}
df_oferta['C_ANIO'] = df_oferta['C_ANIO'].astype(int)
df_oferta['C_MES'] = df_oferta['C_MES'].astype(int)
df_oferta['C_ARTICULO'] = df_oferta['C_ARTICULO'].astype(int)
df_oferta['C_SUCU_EMPR'] = df_oferta['C_SUCU_EMPR'].astype(int)
df_oferta['C_PROVEEDOR_PRIMARIO'] = df_oferta['C_PROVEEDOR_PRIMARIO'].astype(int)


Conectando a SQL Server: 10.54.200.92
Conectando a SQL Server: data-sync


In [7]:
# import pandas as pd
# from datetime import date, datetime, timedelta

# # Función para verificar si la fecha es válida
# def es_fecha_valida(anio, mes, dia):
#     try:
#         return date(anio, mes, dia)
#     except ValueError:
#         return None

# Suponiendo que df_oferta es tu DataFrame original
# Creamos un DataFrame vacío para ir apilando las filas transformadas
df_transformado = pd.DataFrame()

# Iteramos sobre las columnas de ofertas
for col in df_oferta.columns:
    if col.startswith("M_OFERTA_DIA"):
        # Extraemos el número del día de la columna (por ejemplo, 'M_OFERTA_DIA1' -> 1)
        dia = int(col.replace("M_OFERTA_DIA", ""))
        
        # Copiamos los datos necesarios para este día
        df_tmp = df_oferta[['C_ANIO', 'C_MES', 'C_SUCU_EMPR', 'C_PROVEEDOR_PRIMARIO', 'C_ARTICULO', col]].copy()
        df_tmp['DIA'] = dia
        
        # Creamos la fecha a partir del año, mes y día
        df_tmp['FECHA'] = df_tmp.apply(lambda row: es_fecha_valida(row['C_ANIO'], row['C_MES'], row['DIA']), axis=1)
        
        # Filtramos fechas inválidas
        df_tmp = df_tmp.dropna(subset=['FECHA'])
        
        # Filtramos fechas futuras (solo fechas hasta el día anterior)
        df_tmp = df_tmp[df_tmp['FECHA'] <= (datetime.now().date() - timedelta(days=1))]

        # Creamos la columna de 'OFERTA' (1 si tiene oferta, 0 si no)
        df_tmp['OFERTA'] = df_tmp[col].apply(lambda x: 1 if x == 'S' else 0)
        
        # Ahora solo conservamos las columnas relevantes
        df_tmp = df_tmp[['C_PROVEEDOR_PRIMARIO','C_ARTICULO', 'C_SUCU_EMPR', 'FECHA', 'OFERTA']]
        
        # Agregamos al DataFrame final
        df_transformado = pd.concat([df_transformado, df_tmp], ignore_index=True)

# Mostrar el resultado final
print(df_transformado.head())


   C_PROVEEDOR_PRIMARIO  C_ARTICULO  C_SUCU_EMPR       FECHA  OFERTA
0                    20       31872           79  2025-04-01       0
1                    20       31873           79  2025-04-01       0
2                    20       31874           79  2025-04-01       0
3                    20       33091           79  2025-04-01       0
4                    20       33092           79  2025-04-01       0


In [8]:
# Realizamos el merge para agregar las columnas de df_transformado_stock a df_transformado
df_combined = pd.merge(df_transformado, df_trf_stock[['C_PROVEEDOR_PRIMARIO','C_ARTICULO', 'C_SUCU_EMPR', 'FECHA', 'STOCK']], 
                    on=['C_PROVEEDOR_PRIMARIO','C_ARTICULO', 'C_SUCU_EMPR', 'FECHA'], how='left')

# Mostrar el resultado final
print(df_combined.head())


   C_PROVEEDOR_PRIMARIO  C_ARTICULO  C_SUCU_EMPR       FECHA  OFERTA   STOCK
0                    20       31872           79  2025-04-01       0   727.0
1                    20       31873           79  2025-04-01       0  1230.0
2                    20       31874           79  2025-04-01       0    27.0
3                    20       33091           79  2025-04-01       0   128.0
4                    20       33092           79  2025-04-01       0   113.0


In [10]:
fecha_actual = datetime.now().date()
fecha_limite = fecha_actual - timedelta(days=60)
df_filtered = df_combined[df_combined['FECHA'] >= fecha_limite]

## SUBIR DATOS


In [9]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values


def upload_to_postgres(df_combined, table_name="src.historico_ofertas_stock", chunk_size=50000):
    # Conectar a la base de datos
    with open_pg_conn() as conn:
        cur = conn.cursor()

        # Crear la tabla si no existe
        columns = ', '.join(df_combined.columns)
        create_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join([f"{col} {infer_postgres_type(df_combined[col])}" for col in df_combined.columns])}
        );
        """
        cur.execute(create_sql)

        # Insertar los datos en la tabla en bloques (chunks)
        insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES %s"
        total_rows = 0

        for i, chunk in enumerate(range(0, len(df_combined), chunk_size)):
            values_chunk = [tuple(row) for row in df_combined.iloc[chunk:chunk+chunk_size].itertuples(index=False, name=None)]
            execute_values(cur, insert_sql, values_chunk)
            conn.commit()
            total_rows += len(values_chunk)
            print(f"Insertados {len(values_chunk)} registros en el bloque {i+1}")

        # Cerrar cursor
        cur.close()

def infer_postgres_type(series):
    """
    Inferir el tipo de datos PostgreSQL para cada columna del DataFrame
    """
    dtype_map = {
        'int64': 'BIGINT',
        'float64': 'FLOAT',
        'object': 'TEXT',
        'datetime64[ns]': 'TIMESTAMP'
    }
    return dtype_map.get(str(series.dtype), 'TEXT')

# Llamada a la función para subir el DataFrame a PostgreSQL
upload_to_postgres(df_filtered)


Insertados 50000 registros en el bloque 1
Insertados 50000 registros en el bloque 2
Insertados 50000 registros en el bloque 3
Insertados 50000 registros en el bloque 4
Insertados 50000 registros en el bloque 5
Insertados 50000 registros en el bloque 6
Insertados 50000 registros en el bloque 7
Insertados 50000 registros en el bloque 8
Insertados 50000 registros en el bloque 9
Insertados 50000 registros en el bloque 10
Insertados 50000 registros en el bloque 11
Insertados 50000 registros en el bloque 12
Insertados 50000 registros en el bloque 13
Insertados 50000 registros en el bloque 14
Insertados 50000 registros en el bloque 15


KeyboardInterrupt: 

In [13]:
print(ids)

20


In [14]:
# Cargar PRECIOS por Proveedor
"""Consulta el PRECIOS y devuelve un dict {fecha: flag}, limitado a fechas válidas hasta ayer."""
conn = open_sql_conn()
query_precios = f"""
    SELECT  
        P.C_ANIO, P.C_MES,A.C_PROVEEDOR_PRIMARIO, P.C_ARTICULO, P.C_SUCU_EMPR, P.I_PRECIO_VTA_1, P.I_PRECIO_VTA_2, P.I_PRECIO_VTA_3, P.I_PRECIO_VTA_4, P.I_PRECIO_VTA_5 
        ,P.I_PRECIO_VTA_6, P.I_PRECIO_VTA_7, P.I_PRECIO_VTA_8, P.I_PRECIO_VTA_9, P.I_PRECIO_VTA_10, P.I_PRECIO_VTA_11, P.I_PRECIO_VTA_12, P.I_PRECIO_VTA_13 
        ,P.I_PRECIO_VTA_14, P.I_PRECIO_VTA_15, P.I_PRECIO_VTA_16, P.I_PRECIO_VTA_17, P.I_PRECIO_VTA_18, P.I_PRECIO_VTA_19, P.I_PRECIO_VTA_20, P.I_PRECIO_VTA_21  
        ,P.I_PRECIO_VTA_22, P.I_PRECIO_VTA_23, P.I_PRECIO_VTA_24, P.I_PRECIO_VTA_25, P.I_PRECIO_VTA_26, P.I_PRECIO_VTA_27, P.I_PRECIO_VTA_28, P.I_PRECIO_VTA_29 
        ,P.I_PRECIO_VTA_30, P.I_PRECIO_VTA_31 
    FROM [repl].[T710_ESTADIS_PRECIOS] P
    LEFT JOIN [repl].[T050_ARTICULOS] A 
	ON P.C_ARTICULO = A.C_ARTICULO
    WHERE P.C_ANIO = 2025
    AND A.C_PROVEEDOR_PRIMARIO IN ({ids});
"""
df_precios = pd.read_sql(query_precios, conn) # type: ignore
conn.dispose()  # Cerrar conexión SQL Server
if df_stock.empty:
    print(f"⚠️ No se encontraron datos de stock para el proveedor {id_proveedor} en el mes actual.") # type: ignore
    # return {}
df_precios['C_ANIO'] = df_precios['C_ANIO'].astype(int)
df_precios['C_MES'] = df_precios['C_MES'].astype(int)
df_precios['C_ARTICULO'] = df_precios['C_ARTICULO'].astype(int)
df_precios['C_SUCU_EMPR'] = df_precios['C_SUCU_EMPR'].astype(int)

Conectando a SQL Server: 10.54.200.92
Conectando a SQL Server: data-sync


In [ ]:
    # Unir ambos DataFrames por las columnas comunes
    df = pd.merge(df_stock, df_oferta, on=['C_ANIO', 'C_MES', 'C_SUCU_EMPR', 'C_ARTICULO'], how='outer')
    df.fillna(0, inplace=True)  # Rellenar NaN con 0 para las columnas de stock y ofertas

In [ ]:
from datetime import date, datetime, timedelta

def convertir_stock_diario_a_dict(df_stock):
    """Convierte df_stock en un diccionario {fecha: cantidad}, solo hasta ayer y con fechas válidas."""
    def es_fecha_valida(anio, mes, dia):
        try:
            return date(anio, mes, dia)
        except ValueError:
            return None

    resultado = {}
    for _, row in df_stock.iterrows():
        anio = int(row['C_ANIO'])
        mes = int(row['C_MES'])

        for col in df_stock.columns:
            if col.startswith("Q_DIA"):
                dia = int(col.replace("Q_DIA", ""))
                fecha_valida = es_fecha_valida(anio, mes, dia)
                if fecha_valida and fecha_valida <= (datetime.now().date() - timedelta(days=1)):
                    resultado[fecha_valida.isoformat()] = row[col]
    return resultado

# BLOQUE AGREGADO PARA INCORPORAR OFERTAS en formato DICCIONARIO
def convertir_ofertas_a_dict(df_ofertas):
    """Convierte df_ofertas en un diccionario {fecha: flag}, solo hasta ayer y con fechas válidas."""
    def es_fecha_valida(anio, mes, dia):
        try:
            return date(anio, mes, dia)
        except ValueError:
            return None

    resultado = {}
    for _, row in df_ofertas.iterrows():
        anio = int(row['C_ANIO'])
        mes = int(row['C_MES'])

        for col in df_ofertas.columns:
            if col.startswith("M_OFERTA_DIA"):
                dia = int(col.replace("M_OFERTA_DIA", ""))
                fecha_valida = es_fecha_valida(anio, mes, dia)
                if fecha_valida and fecha_valida <= (datetime.now().date() - timedelta(days=1)):
                    resultado[fecha_valida.isoformat()] = row[col]
    return resultado

In [ ]:
datos_stock = convertir_stock_diario_a_dict(df_stock)
datos_ofertas = convertir_ofertas_a_dict(df_oferta)

In [ ]:
print(df_stock.columns)
df_oferta.info()